In [1]:
%config Completer.use_jedi = False

In [2]:
import os
import sys
RunPath = os.getcwd() + "/"
CrysDatPath = "/home/sohamc2/HEA_FCC/MDMC/CrysDat_BCC/"
DataPath = "/home/sohamc2/HEA_FCC/MDMC/ML_runs/DataSets/"
ModulePath = "/home/sohamc2/VKMC/SymNetworkRuns/CE_Symmetry/Symm_Network/"

In [3]:
import numpy as np
import h5py
import torch as pt
from tqdm import tqdm
import pickle
from GCNetRun import Load_Data, makeComputeData, makeProdTensor, Load_crysDats

In [4]:
T = 50
state1List, state2List, dispList, rateList, AllJumpRates, jmpSelects = Load_Data("singleStep_BCC_bin_{}.h5".format(T))

In [6]:
GpermNNIdx, NNsiteList, siteShellIndices, GIndtoGDict, JumpNewSites, dxJumps = Load_crysDats(type="BCC")
print(CrysDatPath, flush=True)
RtoSiteInd = np.load(CrysDatPath + "RtoSiteInd.npy")
SiteIndToR = np.load(CrysDatPath + "SiteIndtoR.npy")

with open(CrysDatPath + "supercellBCC.pkl", "rb") as fl:
    superBCC = pickle.load(fl)
N_units = superBCC.superlatt[0,0]
N_units

/home/sohamc2/HEA_FCC/MDMC/CrysDat_BCC/


8

# first test with single step dataset

In [7]:
dxJumps

array([[ 0.5,  0.5,  0.5],
       [-0.5, -0.5, -0.5],
       [ 0.5, -0.5, -0.5],
       [-0.5,  0.5,  0.5],
       [ 0.5, -0.5,  0.5],
       [-0.5,  0.5, -0.5],
       [ 0.5,  0.5, -0.5],
       [-0.5, -0.5,  0.5]])

In [8]:
specCheck = 1
specsToTrain = [specCheck]
VacSpec = 2
N_check = 200
AllJumps = False
State1_occs, State2_occs, rateData, dispData, OnSites_state1, OnSites_state2 =\
makeComputeData(state1List, state2List, dispList, specsToTrain,
                VacSpec, rateList, AllJumpRates, JumpNewSites,
                dxJumps, NNsiteList, N_check, AllJumps=AllJumps)

OnSiteTensor1 = makeProdTensor(OnSites_state1, 3)
OnSiteTensor2 = makeProdTensor(OnSites_state2, 3)

 37%|███▋      | 148/400 [00:00<00:00, 1476.87it/s]

Building Occupancy Tensors for species : [1]


100%|██████████| 400/400 [00:00<00:00, 1449.21it/s]


In [9]:
dxJumps

array([[ 0.5,  0.5,  0.5],
       [-0.5, -0.5, -0.5],
       [ 0.5, -0.5, -0.5],
       [-0.5,  0.5,  0.5],
       [ 0.5, -0.5,  0.5],
       [-0.5,  0.5, -0.5],
       [ 0.5,  0.5, -0.5],
       [-0.5, -0.5,  0.5]])

In [10]:
for samp in tqdm(range(2*N_check), position=0, leave=True):
    for site in range(state1List.shape[1]):
        if site == 0:
            assert np.all(State1_occs[samp, :, site] == 0)
        else:
            spec1 = state1List[samp, site]
            assert State1_occs[samp, spec1, site] == 1
            assert np.sum(State1_occs[samp, :, site]) == 1
            
            spec2 = state2List[samp, site]
            assert State2_occs[samp, spec2, site] == 1
            assert np.sum(State2_occs[samp, :, site]) == 1
        
    # check the displacements
    jSelect = jmpSelects[samp]
    NNR = np.dot(np.linalg.inv(superBCC.crys.lattice), dxJumps[jSelect]).astype(int) % N_units
    NNsiteVac = NNsiteList[jSelect + 1, 0]
    assert np.all(NNR == SiteIndToR[NNsiteVac])
    spec = state1List[samp, NNsiteVac]
    assert np.allclose(dispData[samp, 0], dxJumps[jSelect])
    if spec == specCheck:
        assert np.allclose(dispData[samp, 1], -dxJumps[jSelect])
    else:
        assert np.allclose(dispData[samp, 1], 0.0)
    
    # check the rate
    assert np.math.isclose(rateData[samp], rateList[samp])

100%|██████████| 400/400 [00:10<00:00, 36.56it/s]


In [11]:
batchStart = 0 #np.random.randint(OnSiteTensor1.shape[0] - N_check)
batchEnd = batchStart + 2*N_check
y = pt.rand(2*N_check, 3, state1List.shape[1]).double()

y1_calc = pt.sum(y*OnSiteTensor1[batchStart : batchEnd], dim=2)
y2_calc = pt.sum(y*OnSiteTensor2[batchStart : batchEnd], dim=2)

for sampInd in tqdm(range(batchStart, batchEnd), position=0, leave=True):
    y1_samp = pt.zeros(3).double()
    y2_samp = pt.zeros(3).double()
    for site in range(OnSiteTensor1.shape[2]):
        occs1 = State1_occs[sampInd, :, site]
        occs2 = State2_occs[sampInd, :, site]
        if occs1[specCheck] == 1:
            y1_samp += y[sampInd - batchStart, :, site]
        if occs2[specCheck] == 1:
            y2_samp += y[sampInd - batchStart, :, site]
    
    assert pt.allclose(y1_calc[sampInd - batchStart], y1_samp)
    assert pt.allclose(y2_calc[sampInd - batchStart], y2_samp)

100%|██████████| 400/400 [00:05<00:00, 71.54it/s]


# now let's try with all jumps

In [12]:
specCheck = 1
specsToTrain = [specCheck]
VacSpec = 2
N_train = 100
AllJumps = True
State1_occs, State2_occs, rateData, dispData, OnSites_state1, OnSites_state2 =\
makeComputeData(state1List, state2List, dispList, specsToTrain,
                VacSpec, rateList, AllJumpRates, JumpNewSites,
                dxJumps, NNsiteList, N_train, AllJumps=AllJumps)

assert State1_occs.shape[0] == State2_occs.shape[0] == rateData.shape[0] == dispData.shape[0] == 1600
assert OnSites_state1.shape[0] == OnSites_state1.shape[0] == 1600

OnSiteTensor1 = makeProdTensor(OnSites_state1, 3)
OnSiteTensor2 = makeProdTensor(OnSites_state2, 3)

  9%|▉         | 18/200 [00:00<00:01, 179.45it/s]

Building Occupancy Tensors for species : [1]


100%|██████████| 200/200 [00:01<00:00, 178.03it/s]


In [13]:
State1_occs.shape[0]

1600

In [15]:
for stateInd in tqdm(range(N_train*2), position=0, leave=True):
    state1 = state1List[stateInd]
    for jInd in range(dxJumps.shape[0]):
        state2 = state1.copy()
        NNsiteVac = NNsiteList[jInd + 1, 0]
        RsiteVac = SiteIndToR[NNsiteVac]
        assert state2[0] == 2
        state2[0] = state2[NNsiteVac]
        state2[NNsiteVac] = 2
        
        for site in range(state1List.shape[1]):
            Rsite = SiteIndToR[site]
            RsiteNew = (Rsite + RsiteVac) % N_units
            siteNew = RtoSiteInd[RsiteNew[0], RsiteNew[1], RsiteNew[2]]
            spec1 = state1[site]
            spec2 = state2[siteNew]
            
            if site == 0:
                assert spec2 == 2
                assert np.all(State1_occs[stateInd * dxJumps.shape[0] + jInd, :, site] == 0)
                assert np.all(State2_occs[stateInd * dxJumps.shape[0] + jInd, :, site] == 0)
            
            else:
                assert spec2 != 2
                assert State1_occs[stateInd * dxJumps.shape[0] + jInd, spec1, site] == 1
                assert np.sum(State1_occs[stateInd * dxJumps.shape[0] + jInd, :, site]) == 1

                assert State2_occs[stateInd * dxJumps.shape[0] + jInd, spec2, site] == 1
                assert np.sum(State2_occs[stateInd * dxJumps.shape[0] + jInd, :, site]) == 1
        
        # check the displacements
        NNR = np.dot(np.linalg.inv(superBCC.crys.lattice), dxJumps[jInd]).astype(int) % N_units
        NNsiteVac = NNsiteList[jInd + 1, 0]
        assert np.all(NNR == SiteIndToR[NNsiteVac])
        spec = state1[NNsiteVac]
        assert np.allclose(dispData[stateInd * dxJumps.shape[0] + jInd, 0], dxJumps[jInd])
        if spec == specCheck:
            assert np.allclose(dispData[stateInd * dxJumps.shape[0] + jInd, 1], -dxJumps[jInd])
        
        # check the rate
        assert np.allclose(rateData[stateInd * dxJumps.shape[0] + jInd], AllJumpRates[stateInd, jInd])

100%|██████████| 200/200 [01:01<00:00,  3.26it/s]


In [16]:
np.linalg.norm(dispList[0, VacSpec, :])/np.linalg.norm(dxJumps[0])

1.0

In [18]:
batchStart = 0 #np.random.randint(OnSiteTensor1.shape[0] - 1000)
batchEnd = batchStart + 1000
y = pt.rand(1000, 3, state1List.shape[1]).double()

y1_calc = pt.sum(y*OnSiteTensor1[batchStart : batchEnd], dim=2)
y2_calc = pt.sum(y*OnSiteTensor2[batchStart : batchEnd], dim=2)

for sampInd in tqdm(range(batchStart, batchEnd), position=0, leave=True):
    y1_samp = pt.zeros(3).double()
    y2_samp = pt.zeros(3).double()
    for site in range(OnSiteTensor1.shape[2]):
        occs1 = State1_occs[sampInd, :, site]
        occs2 = State2_occs[sampInd, :, site]
        if occs1[specCheck] == 1:
            y1_samp += y[sampInd - batchStart, :, site]
        if occs2[specCheck] == 1:
            y2_samp += y[sampInd - batchStart, :, site]
    
    assert pt.allclose(y1_calc[sampInd - batchStart], y1_samp)
    assert pt.allclose(y2_calc[sampInd - batchStart], y2_samp)

100%|██████████| 1000/1000 [00:13<00:00, 71.78it/s]
